In [1]:
# Install required packages if not already present
import subprocess
import sys

def install_package(package):
    """Install a package using pip if not already installed."""
    try:
        __import__(package.replace('-', '_').split('[')[0])
        print(f"✓ {package} already installed")
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✓ {package} installed successfully")

# List of required packages from requirements.txt
required_packages = [
    "langchain",
    "langchain-community",
    "ollama",
    "chromadb",
    "sentence-transformers",
    "pypdf",
    "unstructured",
    "python-dotenv",
    "tqdm"
]

print("Checking and installing required packages...")
for package in required_packages:
    install_package(package)

Checking and installing required packages...
✓ langchain already installed
✓ langchain-community already installed
✓ ollama already installed
✓ chromadb already installed


/home/oscar/playground/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ sentence-transformers already installed
✓ pypdf already installed
✓ unstructured already installed
Installing python-dotenv...
✓ python-dotenv installed successfully
✓ tqdm already installed


In [2]:
import langchain
print(langchain.__version__)


1.2.0


In [3]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Make sure ollama is running and llama3 is pulled:
#   ollama serve
#   ollama pull llama3

llm = ChatOllama(model="llama3", temperature=0.0, seed=42)

# Quick test
print(llm.invoke("Say hello in one short sentence.").content)


/tmp/ipykernel_65536/3847789316.py:10: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3", temperature=0.0, seed=42)


Hello!


In [17]:
# Invoke with a question about green thai curry
msg = llm.invoke("What are the ingredients of green thai curry.")
print("Type:", type(msg))

# List simple attributes (excluding methods and privates)
attrs = [a for a in dir(msg) if not a.startswith("_") and not callable(getattr(msg, a))]
print("Attributes:", attrs)

# Safe dump of the whole message (Pydantic v2 -> model_dump; v1 -> dict)
try:
    dump = msg.model_dump()
except Exception:
    dump = msg.dict()

print("\nDump keys:", list(dump.keys()))
print("\ncontent:\n", dump.get("content"))
print("\nresponse_metadata:\n", dump.get("response_metadata"))
print("\nadditional_kwargs:\n", dump.get("additional_kwargs"))
print("\nid:\n", dump.get("id"))
print("\ntool_calls:\n", dump.get("tool_calls"))
print("\ntype field:\n", dump.get("type"))

Type: <class 'langchain_core.messages.ai.AIMessage'>
Attributes: ['additional_kwargs', 'content', 'content_blocks', 'id', 'invalid_tool_calls', 'lc_attributes', 'lc_secrets', 'model_computed_fields', 'model_config', 'model_extra', 'model_fields', 'model_fields_set', 'name', 'response_metadata', 'tool_calls', 'type', 'usage_metadata']

Dump keys: ['content', 'additional_kwargs', 'response_metadata', 'type', 'name', 'id', 'tool_calls', 'invalid_tool_calls', 'usage_metadata']

content:
 Delicious Green Thai Curry! Here are the common ingredients you'll find in a traditional recipe:

**Curry Paste:**

* Green Thai curry paste (made from a blend of spices, including lemongrass, galangal, chilies, garlic, shallots, and turmeric)

**Protein:**

* Chicken or shrimp (common protein choices)
* You can also use tofu or vegetables for a vegetarian option

**Aromatics:**

* Onions
* Garlic
* Shallots
* Lemongrass
* Galangal (Thai ginger)

**Coconut Milk:**

* Full-fat coconut milk adds richness and

/tmp/ipykernel_14044/1850993692.py:6: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  attrs = [a for a in dir(msg) if not a.startswith("_") and not callable(getattr(msg, a))]
/tmp/ipykernel_14044/1850993692.py:6: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  attrs = [a for a in dir(msg) if not a.startswith("_") and not callable(getattr(msg, a))]


In [22]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are concise. Answer in one sentence."),
    ("user", "{question}")
])

inputs = {"question": "What are nice ingredients in summer rolls?"}
resp = (prompt | llm).invoke(inputs)
print(resp.content)

print("Input variables:", prompt.input_variables)
for var in prompt.input_variables:
    print(f"- {var}", repr(inputs.get(var)))

formatted_msgs = prompt.format_messages(question="What is prompt engineering?")
print("\nFormatted messages:")
for m in formatted_msgs:
    print(f"- role={m.type} content={repr(m.content)}")

Nice ingredients to include in summer rolls typically consist of fresh vegetables like shrimp, carrots, cucumbers, and avocado, combined with herbs like mint and basil, and wrapped in a thin rice paper wrapper.
Input variables: ['question']
- question 'What are nice ingredients in summer rolls?'

Formatted messages:
- role=system content='You are concise. Answer in one sentence.'
- role=human content='What is prompt engineering?'


In [23]:
from langchain_core.output_parsers import StrOutputParser
# 3) Manual invocation: pass messages directly to the LLM
manual_resp = llm.invoke(formatted_msgs)
print("\nManual resp type:", type(manual_resp))
print("Manual resp content:", manual_resp.content)

# 4) Piped invocation: (prompt | llm).invoke(...)
piped_chain = prompt | llm
piped_resp = piped_chain.invoke({"question": "What is prompt engineering?"})
print("\nPiped resp type:", type(piped_resp))
print("Piped resp content:", piped_resp.content)

# 5) If you want a plain string instead of an AIMessage, add a string parser
text_chain = prompt | llm | StrOutputParser()
text_out = text_chain.invoke({"question": "What is prompt engineering?"})
print("\nParsed text:", text_out)



Manual resp type: <class 'langchain_core.messages.ai.AIMessage'>
Manual resp content: Prompt engineering is the process of designing and optimizing natural language prompts to elicit specific, accurate, and relevant responses from language models or other AI systems.

Piped resp type: <class 'langchain_core.messages.ai.AIMessage'>
Piped resp content: Prompt engineering is the process of designing and optimizing natural language prompts to elicit specific, accurate, and relevant responses from language models or other AI systems.

Parsed text: Prompt engineering is the process of designing and optimizing natural language prompts to elicit specific, accurate, and relevant responses from language models or other AI systems.


In [ ]:
# Minimal corpus; replace with your own texts or load a file

texts = [
    "LangChain helps build LLM applications. RAG steps: ingest documents, embed/index, retrieve relevant chunks, answer with citations.",
    "Few-shot prompting provides examples to steer the model toward desired output formats and styles."
]
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vs = Chroma.from_texts(texts, embedding=embeddings, collection_name="mini")
retriever = vs.as_retriever(search_kwargs={"k": 2})

def ask_rag(question: str) -> str:
    docs = retriever.invoke(question)  # changed from get_relevant_documents(...)
    context = "\n\n".join(d.page_content for d in docs)
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "Answer based only on the provided context. If the answer is not in the context, say you don't know."),
        ("user", "Context:\n{context}\n\nQuestion: {question}")
    ])
    return (rag_prompt | llm).invoke({"context": context, "question": question}).content

print(ask_rag("What are the steps of a RAG pipeline?"))

print(ask_rag("What is few-shot prompting?"))

print(ask_rag("What are known drum and bass artists?"))


According to the provided context, the steps of a RAG (Reinforced Answer Generation) pipeline are:

1. Ingest documents
2. Embed/index
3. Retrieve relevant chunks
4. Answer with citations
Few-shot prompting provides examples to steer the model toward desired output formats and styles.
I don't know. The provided context does not mention drum and bass artists. It only talks about few-shot prompting and its application in steering models towards desired output formats and styles.
What are known drum and bass artists?


In [24]:
print(llm.invoke("What are known drum and bass artists?").content)

Drum and bass is a popular electronic music genre that has spawned many talented artists over the years. Here's a list of some well-known drum and bass artists:

1. Pendulum: A pioneering group in the drum and bass scene, known for their energetic live performances and hits like "Slam" and "Tarantula".
2. Sub Focus: A British producer and DJ who has been a major force in the genre since the early 2000s, with tracks like "Rock It" and "Follow the Sun".
3. Andy C: A legendary DJ and producer who has been a driving force behind the drum and bass scene for over two decades, with hits like "Heartbeat Loud" and "Quest".
4. Netsky: A Belgian producer and DJ known for his melodic and atmospheric sound, with tracks like "Puppy" and "We Can Only Imagine".
5. Camo & Krooked: An Austrian production duo that has gained a reputation for their unique blend of drum and bass and dubstep, with hits like "Move Around" and "All Night".
6. Sigma: A British production duo that has had significant success in

In [25]:
print("\nConfigured num_ctx:", llm.num_ctx)
print("Configured num_predict:", llm.num_predict)


Configured num_ctx: None
Configured num_predict: None


In [4]:
role = "You are an experienced chef, knowing a lot about ingredients and recipes."
prompt = ChatPromptTemplate.from_messages([
    ("system", f"{role}"),
    ("user", "{question}")
])

inputs = {"question": """
            Which ingredients are typically used in a traditional thai red curry?
          """}
resp = (prompt | llm).invoke(inputs)
print(resp.content)

Traditional Thai Red Curry is a staple dish in Thai cuisine! It's a flavorful and spicy curry made with a variety of ingredients. Here are the typical ones you'll find:

1. **Red Curry Paste**: This is the foundation of the curry, made from a blend of spices like chilies, garlic, lemongrass, galangal, cumin, coriander, and turmeric.
2. **Coconut Milk**: Rich and creamy coconut milk adds depth and richness to the curry. You can use full-fat or low-fat coconut milk, depending on your preference.
3. **Chilies**: Fresh or dried chilies like bird's eye chilies, Thai chilies, or jalapeños add heat to the curry. You can adjust the level of spiciness to your liking.
4. **Onions**: Thinly sliced or diced onions sautéed with garlic and ginger form the base of the curry.
5. **Ginger**: Fresh or ground ginger adds warmth and depth to the curry.
6. **Garlic**: Like onions, garlic is sautéed with ginger to create a flavorful foundation for the curry.
7. **Protein**: You can use various proteins like